**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Nov 24 06:10:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov 24 06:11:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'GN - Life'	'My Drive'   yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15368, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 15368 (delta 0), reused 1 (delta 0), pack-reused 15367
Receiving objects: 100% (15368/15368), 13.98 MiB | 19.77 MiB/s, done.
Resolving deltas: 100% (10335/10335), done.


# 새 섹션

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
!cp /mydrive/yolov3/data/yolov3_testing.cfg cfg/yolov3_training.cfg

In [ ]:
!echo -e '' > data/obj.names
!echo -e '' > data/obj.data
!cp /mydrive/yolov3/data/obj.names data/obj.names
!cp /mydrive/yolov3/data/obj.data data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-11-24 06:12:57--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  78.7MB/s    in 2.0s    

2021-11-24 06:12:59 (78.7 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: data/obj/helmet/S2-N1802M13790.jpg  
  inflating: data/obj/helmet/S2-N1802M13790.txt  
 extracting: data/obj/helmet/S2-N1802M13792.jpg  
  inflating: data/obj/helmet/S2-N1802M13792.txt  
 extracting: data/obj/helmet/S2-N1802M13793.jpg  
  inflating: data/obj/helmet/S2-N1802M13793.txt  
  inflating: data/obj/helmet/S2-N1802M13873.jpg  
  inflating: data/obj/helmet/S2-N1802M13873.txt  
  inflating: data/obj/helmet/S2-N1802M13875.jpg  
  inflating: data/obj/helmet/S2-N1802M13875.txt  
  inflating: data/obj/helmet/S2-N1802M13876.jpg  
  inflating: data/obj/helmet/S2-N1802M13876.txt  
 extracting: data/obj/helmet/S2-N1802M13980.jpg  
  inflating: data/obj/helmet/S2-N1802M13980.txt  
 extracting: data/obj/helmet/S2-N1802M13981.jpg  
  inflating: data/obj/helmet/S2-N1802M13981.txt  
 extracting: data/obj/helmet/S2-N1802M13982.jpg  
  inflating: data/obj/helmet/S2-N1802M13982.txt  
 extracting: data/obj/helmet/S2-N1802M13983.jpg  
  inflating: d

In [ ]:
import glob
images_list = glob.glob("data/obj/helmet/*.jpg")
#images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/helmet/S2-N1803M00556.jpg', 'data/obj/helmet/S2-N1803M00242.jpg', 'data/obj/helmet/S2-N1802M14391.jpg', 'data/obj/helmet/hard_hat_workers3315.jpg', 'data/obj/helmet/S2-N1001M02480.jpg', 'data/obj/helmet/hard_hat_workers3440.jpg', 'data/obj/helmet/S2-N2301M00945.jpg', 'data/obj/helmet/hard_hat_workers2156.jpg', 'data/obj/helmet/S2-N1001M02507.jpg', 'data/obj/helmet/hard_hat_workers435.jpg', 'data/obj/helmet/S2-N1802M12380.jpg', 'data/obj/helmet/S2-N1802M12796.jpg', 'data/obj/helmet/S2-N1803M15020.jpg', 'data/obj/helmet/hard_hat_workers1191.jpg', 'data/obj/helmet/S2-N1802M17507.jpg', 'data/obj/helmet/hard_hat_workers2989.jpg', 'data/obj/helmet/S2-N1803M20208.jpg', 'data/obj/helmet/hard_hat_workers1812.jpg', 'data/obj/helmet/S2-N1802M09963.jpg', 'data/obj/helmet/hard_hat_workers381.jpg', 'data/obj/helmet/S2-N2301M00090.jpg', 'data/obj/helmet/S2-N1803M02188.jpg', 'data/obj/helmet/S2-N2001M14249.jpg', 'data/obj/helmet/hard_hat_workers1099.jpg', 'data/obj/helmet/S2-N1803M00392.jpg

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000057, iou_loss = 0.000000, total_loss = 0.000057 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.819451), count: 7, class_loss = 0.324518, iou_loss = 0.164275, total_loss = 0.488793 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.654523), count: 9, class_loss = 0.734537, iou_loss = 0.386495, total_loss = 1.121032 
 total_bbox = 283500, rewritten_bbox = 0.093474 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.544842), count: 2, class_loss = 0.090144, iou_loss = 0.155444, total_loss = 0.245588 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.735370), count: 15, class_loss = 1.414961, iou_loss = 0.579321, total_loss = 1.994282 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 